In [ ]:
from sklearn import datasets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = datasets.load_iris()

In [ ]:
df

In [ ]:
df.data

In [ ]:
df.target

In [ ]:
df.feature_names

In [ ]:
details = {
    'Sepal Length' : df.data[:,0],
    'Sepal Width' : df.data[:,1],
    'Petal Length' : df.data[:,2],
    'Petal Width' : df.data[:,3],
    'Species' : df.target
}

In [ ]:
df2 = pd.DataFrame(details)
df2

In [ ]:
df2['Species'] = df2['Species'].apply(lambda x:df.target_names[x])

In [ ]:
df2

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = df2['Species']
y

In [ ]:
x = df2.drop(['Species'], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.25)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
from sklearn import svm

In [ ]:
model = svm.SVC(kernel = 'rbf', C=30, gamma ='auto')
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
model2 = svm.SVC(kernel = 'linear', C=30, gamma ='auto')
model2.fit(X_train, y_train)
model2.score(X_test, y_test)

In [ ]:
model3 = svm.SVC(kernel = 'poly', C=30, gamma ='auto')
model3.fit(X_train, y_train)
model3.score(X_test, y_test)

In [ ]:
model4 = svm.SVC(kernel = 'sigmoid', C=30, gamma ='auto')
model4.fit(X_train, y_train)
model4.score(X_test, y_test)

### KFold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross1 = cross_val_score(svm.SVC(kernel = 'rbf', C=30, gamma ='auto'),x ,y, cv = 5)
cross1

In [ ]:
cross2 = cross_val_score(svm.SVC(kernel = 'linear', C=30, gamma ='auto'),x ,y, cv = 5)
cross2

In [ ]:
cross3 = cross_val_score(svm.SVC(kernel = 'poly', C=30, gamma ='auto'),x ,y, cv = 5)
cross3

In [ ]:
print(np.average(cross1)) # rbf
print(np.average(cross2)) # linear
print(np.average(cross3)) # poly

In [ ]:
ker = ['rbf', 'linear', 'poly']
C = [1,10,20]
total_score = {}

for k in ker:
    for cval in C:
        cv_score = cross_val_score(svm.SVC(kernel = k, C = cval, gamma ='auto'), x, y, cv = 5)
        total_score[k+ '_' +str(cval)] = np.average(cv_score)
        

In [ ]:
total_score

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
gs = GridSearchCV(svm.SVC(gamma='auto'), {'C': [1,10,20], 'kernel' : ['rbf', 'linear', 'poly']}, 
                  cv = 5, return_train_score= False)

In [ ]:
gs.fit(x,y)

In [ ]:
gs.cv_results_

In [ ]:
data_gs = pd.DataFrame(gs.cv_results_)

In [ ]:
data_gs

In [ ]:
data_gs = data_gs[['param_C', 'param_kernel', 'mean_test_score']]

In [ ]:
data_gs

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rs = RandomizedSearchCV(svm.SVC(gamma = 'auto'), {'C' : [1,10,20], 'kernel': ['rbf', 'linear', 'poly']}, cv = 5, return_train_score = False, n_iter = 2)

In [ ]:
rs.fit(x,y)

In [ ]:
rs.cv_results_

In [ ]:
data_rs = pd.DataFrame(rs.cv_results_)

In [ ]:
data_rs = data_rs[['param_C', 'param_kernel', 'mean_test_score']]
data_rs

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model_parm = {
    'svm': {
        'model': svm.SVC(gamma ='auto'), 
        'params': {
            'C': [1,10,20],
                    'kernel': ['rbf', 'linear', 'poly']
        }
    }, 'RandomForest': {
        'model': RandomForestClassifier(), 
        'params': {'n_estimators': [1,5,10]
                   }
        
    }, 
    'Logistic_regression' : {
        'model': LogisticRegression(), 
        'params': {
            'C': [1,10,20]
        }
    }
}

In [ ]:
scores=[]
for model_name,mp in model_parm.items():
    cl=GridSearchCV (mp['model'],mp['params'],cv=5,return_train_score=False)
    cl.fit(x,y)
    scores.append({
        'model':model_name,
        "best_score":cl.best_score_,
        "best_parms":cl.best_params_
    })


In [ ]:
d=pd.DataFrame(scores,columns=['model','best_score','best_parms'])

In [ ]:
d